<a href="https://colab.research.google.com/github/IshantWadhwa4/MLFramework/blob/master/CommonCodeFrameWork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualization

In [0]:
# plot box graph to check if there any outlier or not

#import seaborn as sns
#import matplotlib.pyplot as plt
def plot_Outlier_BoxPlot(df):
  for col in df.columns:
    sns.boxplot(y=col,data=df)
    plt.show()


ERROR! Session/line number was not unique in database. History logging moved to new session 59


In [0]:
# Plot corelation b/w continous variables
def plot_correlation_heatmap(df):
  corr_df = df.corr()
  sns.heatmap(corr_df)
  plt.show()

In [0]:
# scatter plot between given columns
# import plotly_express as px

def plot_scatter_plotly(df,list_columns=[]):
  if list_columns=[] :
     px.scatter_matrix(df)
  else:
    px.scatter_matrix(df[list_columns])


In [0]:
# create Heatmap for highly co-related(given threshold) columns
# seaborn pandas and numpy

def create_seaborn_heatmap(df,posThreshold,negThreshold):
  '''
      create Heatmap for highly co-related(given threshold) columns

      Input: dataframe, positive threshold, negitive threshold
      Plot: Heatmap
  '''
  tempdf = df[(df > posThreshold) | (df < -negThreshold)]
  tempdf.replace(to_replace=1,value=np.nan,inplace=True)
  tempdf.dropna(axis=1,how='all',inplace=True)
  tempdf.dropna(axis=0,how='all',inplace=True)
  sns.heatmap(tempdf)




# Pre Modeling functions

In [0]:
# if data is large and you want sample from it 

def data_sampling(df,sample_size):
  '''
      you want sample from data frame.

      Input: df is data frame
            sample_size id how much % of data you want if you want 50% than it is .5
      output: df of sample size
 
  '''
  return df.sample(frac = sample_size).reset_index(drop=True)



In [0]:
# LIb : need pandas lib for this function.
# Parameters : Only required dataframe for which you want zeros and null for each column
# return: dataframe with number of zeros and null

def get_number_zeros_null(df):
  '''
       LIb : need pandas lib for this function.

       Input : Only required dataframe for which you want zeros and null for each column       
       Output: dataframe with number of zeros and null
  '''
  null_zero_dict={ }
  null_zero_dict['Number_of_nulls'] = df.isnull().sum()
  null_zero_dict['Number_of_zeros'] = (df==0).astype(int).sum()
  return pd.DataFrame(null_zero_dict).T


In [0]:
# Get count of class in given catagorical column. Reqire Pandas lib.
def get_count_class_catagColumn(column_values):
  '''
    Get count of class in given catagorical column. Reqire Pandas lib.

    Input:  column values(df['columnName'])
    output: Pandas series object with count of each class in the column.
            
  '''
  return pd.value_counts(column_values)

In [0]:
# convert type of columns

def convert_type(df,list_column,list_type):
  '''
    convert column's data type
    
    Input: dataframe, list of columns you want to convert, type in which you want to convert
    output: pandas data frame   
  '''
  for k,col in enumerate(list_column):
    df[col] = df[col].astype(list_type[k])
  return df

In [0]:
# In classification problem if number of classes are unbalanced



# Model Prepration

### Model Spliting

In [0]:

from sklearn.model_selection import train_test_split

def model_split_train_test(df,target_column_name,test_size):
  X = df.drop(target_column_name,axis=1)
  y = df[target_column_name]
  return train_test_split(X,y,test_size = test_size,random_state = 42,stratify = y)




In [0]:
from sklearn.model_selection import ShuffleSplit

def model_shuffleSplit_train_test(splits = 10,test_size=.3):
  return ShuffleSplit(n_splits = splits, test_size = test_size, train_size = 1-test_size, random_state = 42 )

### Model Encoding

### Model Selection

In [0]:
# fiting train data in model
from sklearn.model_selection import ShuffleSplit,cross_validate


def get_model_predictions_result(model_object,X,y):
  #set name and parameters
  final_result = {}
  final_result['Algo_name'] = model_object.__class__.__name__
  cv_results = model_selection.cross_validate(model_object, X, y, cv  = model_shuffleSplit_train_test(10,.25),return_train_score=True)
  final_result['Time'] = cv_results['fit_time'].mean()
  final_result['Train Accuracy'] = cv_results['train_score'].mean()
  final_result['Test Accuracy'] = cv_results['test_score'].mean()   
  #if this is a non-bias random sample, then +/-3 standard deviations (std) from the mean, should statistically capture 99.7% of the subsets
  final_result['Test Accuracy 3*STD'] = cv_results['test_score'].std()*3
  return final_result



In [0]:
from sklearn.metrics import confusion_matrix,auc,roc_auc_score, classification_report
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score

def print_scores(y_test,y_pred,y_pred_prob):
    print('test-set confusion matrix:\n', confusion_matrix(y_test,y_pred)) 
    print ("The Classification Report: \n", classification_report(y_test, y_pred))
    print("recall score: ", recall_score(y_test,y_pred))
    print("precision score: ", precision_score(y_test,y_pred))
    print("f1 score: ", f1_score(y_test,y_pred))
    print("accuracy score: ", accuracy_score(y_test,y_pred))
    print("ROC AUC: {}".format(roc_auc_score(y_test, y_pred_prob[:,1])))

In [0]:
from sklearn.metrics import confusion_matrix,auc,roc_auc_score, classification_report

def get_model_confusion_matrix(model_object,df,target_column,test_size = .25):
  X_train,X_test,y_train,y_test = model_split_train_test(df,target_column,test_size)
  result = model_object.fit(X_train,y_train)
  y_pred = result.predict(X_test)
  y_pred_prob = result.predict_proba(X_test)
  return (y_test,y_pred,y_pred_prob) 
  

### For Supervise learning

In [0]:
# Stratified K-Folds cross-validator for cross validation and make a train kfold csv file
# for fold 0 some values(random) will be treated as test and remaing as train
# for fold 1 some values(random) will be treated as test and remaing as train  and so on kfold column significants that
# this is for taking control to your self


from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold( n_splits=5, shuffle=True, random_state= 42 )

for fold,(train_idx,test_idx) in enumerate(kfold.split(X = df.drop('Target_column',axis=1),y=df['Target_column'])):
  print(fold,len(train_idx),len(test_idx))
  df.loc[test_idx,'kfold'] = fold

df.to_csv("train_fold.csv", index = False) 
 



# Post Model Functions